# Object Detection using Amazon SageMaker (Incremental Training)

In this example, we will show you how to train an object detector by **re-using a model you previously trained in the SageMaker**. With this model re-using ability, you can save the training time when you update the model with new data or improving the model quality with the same data. In the first half of this notebook (Intial Training), we will follow the training with RecordIO format example to train a object detection model on the Pascal VOC dataset. In the second half, we will show you how you can re-use the trained model and improve its quality without repeating the entire training process.

### Steps needed before the initial training are basically identical to 'Object Detection using Amazon SageMaker (RecordIO format)' notebook.
### Please refer to that notebook for detailed explanations.

In [ ]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import time
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import boto3
import botocore
from botocore.exceptions import ClientError

import csv
import io
import re
import s3fs
import mxnet as mx
import seaborn as sns
import pickle
import gzip
import urllib
import csv


import sagemaker                                 
from sagemaker.predictor import csv_serializer 
from sagemaker.predictor import json_deserializer
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role

### 1. Preparation (Specifying Sagemaker roles)

In [ ]:
sess   = sagemaker.Session()
bucket = sess.default_bucket()                     
prefix = 'objectdetection'
region = boto3.Session().region_name
role = 'arn:aws:iam::570447867175:role/SageMakerNotebookRole' # pass your IAM role name

print('Sagemaker session :', sess)
print('S3 bucket :', bucket)
print('Prefix :', prefix)
print('Region selected :', region)
print('IAM role :', role)

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(region, 'object-detection', repo_version="latest")
print(training_image)

#### Follow the steps in 'Object Detection using Amazon SageMaker (RecordIO format)' notebook

In [ ]:
### 2. Load Data

In [ ]:
### 3. UpLoad Data to S3 

In [ ]:
### 4. Create a Model

In [ ]:
### 5. Start Training

In [ ]:
# od_model.fit(inputs=data_channels, logs=True)

#### After we fitting the model, we can start a new training job from the model obtained in previous section. We setup the estimator and hyperparameters similar to the previous training job. Note that SageMaker object detection algorithm currently only support the re-training feature with the same network, which means the new training job must have the same base_network and num_classes as the previous training job.

In [ ]:
new_od_model = sagemaker.estimator.Estimator(training_image,
                                             role, 
                                             train_instance_count=1, 
                                             train_instance_type='ml.p3.2xlarge',
                                             train_volume_size = 50,
                                             train_max_run = 360000,
                                             input_mode= 'File',
                                             output_path=s3_output_location,
                                             sagemaker_session=sess)

In [ ]:
new_od_model.set_hyperparameters(base_network='resnet-50',
                                 num_classes=20,
                                 mini_batch_size=16,
                                 epochs=1,
                                 learning_rate=0.001,
                                 optimizer='rmsprop',
                                 momentum=0.9,
                                 image_shape=300,
                                 label_width=350,
                                 num_training_samples=16551)

#### We should use the same training data from previous job
To use the pre-trained model, we just need to add a model channel to the inputs and set its content type to application/x-sagemaker-model.

In [ ]:
# Use the same data for training and validation as the previous job.
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='application/x-recordio', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='application/x-recordio', s3_data_type='S3Prefix')

# Use the output model from the previous job.  
s3_model_data = od_model.model_data

model_data = sagemaker.session.s3_input(s3_model_data, distribution='FullyReplicated', 
                             content_type='application/x-sagemaker-model', s3_data_type='S3Prefix')

# In addition to two data channels, add a 'model' channel for the training.
new_data_channels = {'train': train_data, 'validation': validation_data, 'model': model_data}

Fit the new model with all three input channels.

In [ ]:
new_od_model.fit(inputs=new_data_channels, logs=True)

In [ ]:
## Model Deply and Inference also resembles the previous notebook